In [12]:
import face_recognition
import os
import pickle

In [25]:
def generate_encodings(folder_path):

    all_files = os.listdir(folder_path)

    # Filter for image files (e.g., .jpg, .png, .jpeg)
    img_path = [filename for filename in all_files if filename.lower().endswith(('.jpg', '.png', '.jpeg', '.gif', '.bmp'))]

    face_encodings = []
    face_name = []

    for i in img_path:
        picture_of_me = face_recognition.load_image_file(f"./photos/{i}")
        face_encodings.append(face_recognition.face_encodings(picture_of_me)[0])
        face_name.append(i.split('.')[0].strip())
        
        
    with open('face_encodings', 'wb') as file:
        pickle.dump(face_encodings, file)

    with open('people', 'wb') as file:
        pickle.dump(face_name, file)

generate_encodings("./photos")

In [27]:
def load_encondings(face_encodings_pkl_path: str, people_pkl_path: str) -> list:

    with open(face_encodings_pkl_path, 'rb') as file:
        face_encodings = pickle.load(file)
    
    with open(people_pkl_path, 'rb') as file:
        face_name = pickle.load(file)

    return face_encodings, face_name

load_encondings('./face_encodings', './people')

([array([-0.13700375,  0.16100572,  0.04092748, -0.02365841, -0.11801755,
         -0.08351278, -0.1408792 , -0.18044651,  0.08126658, -0.08691923,
          0.26213771, -0.02889964, -0.2421007 , -0.13221981, -0.05089916,
          0.11634233, -0.19576026, -0.12375657,  0.00575543, -0.08637879,
          0.12693813,  0.04480563,  0.10622787,  0.01097302, -0.08546417,
         -0.41749287, -0.10531054, -0.13547097,  0.00560574, -0.08188238,
          0.01915912,  0.00715153, -0.168384  , -0.0201347 , -0.00969316,
          0.027528  ,  0.01547773,  0.00998024,  0.14478831,  0.00569959,
         -0.18699473,  0.00166842,  0.0062689 ,  0.30306154,  0.15190987,
          0.06121295,  0.02755924, -0.13273275,  0.10765381, -0.14419641,
          0.09959362,  0.17760253,  0.07965053, -0.01387945,  0.05709012,
         -0.0511117 ,  0.04596403,  0.09806386, -0.16297744,  0.03311126,
          0.15247007, -0.04119916, -0.08358599, -0.05342638,  0.27148071,
          0.06136508, -0.1389657 , -0.

In [33]:
import face_recognition
import cv2
import numpy as np

def load_encondings(face_encodings_pkl_path: str, people_pkl_path: str) -> list:

    with open(face_encodings_pkl_path, 'rb') as file:
        face_encodings = pickle.load(file)
    
    with open(people_pkl_path, 'rb') as file:
        face_name = pickle.load(file)

    return face_encodings, face_name



known_face_encodings, known_face_names = load_encondings('./face_encodings', './people')

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

video_capture = cv2.VideoCapture(0)

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Only process every other frame of video to save time
    if process_this_frame:
        # Resize frame of video to 1/4 size for faster face recognition processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = small_frame[:, :, ::-1]
        
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            # # If a match was found in known_face_encodings, just use the first one.
            # if True in matches:
            #     first_match_index = matches.index(True)
            #     name = known_face_names[first_match_index]

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            face_names.append(name)

    process_this_frame = not process_this_frame


    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()

In [14]:
import mysql.connector
import pandas as pd

test = mysql.connector.connect(
    host='sql12.freesqldatabase.com',
    user='sql12644747',
    passwd='wtivVftCdI'
)

test_cursor = test.cursor()
test_cursor.execute("show databases;")
dbs = test_cursor.fetchall()

test_cursor.close()
test.close()

dbs

[('information_schema',), ('sql12644747',)]

In [16]:
db = mysql.connector.connect(
    host='sql12.freesqldatabase.com',
    user='sql12644747',
    passwd='wtivVftCdI',
    database='sql12644747'
)
mycursor = db.cursor()

mycursor.execute("""
show tables
""")
result = mycursor.fetchall()

mycursor.close()
db.close()

result

[('attendence',)]

In [18]:
db = mysql.connector.connect(
    host='sql12.freesqldatabase.com',
    user='sql12644747',
    passwd='wtivVftCdI',
    database='sql12644747'
)
mycursor = db.cursor()

mycursor.execute("""
describe attendence
""")
result = mycursor.fetchall()

mycursor.close()
db.close()

result

[('name', 'varchar(20)', 'NO', '', None, ''),
 ('number', 'int(10)', 'NO', '', None, '')]

In [29]:
db = mysql.connector.connect(
    host='sql12.freesqldatabase.com',
    user='sql12644747',
    passwd='wtivVftCdI',
    database='sql12644747'
)
mycursor = db.cursor()

mycursor.execute("""
INSERT INTO attendence (name, number)
VALUES ('Sudeep', 6969);
""")
db.commit()

mycursor.close()
db.close()

# result

In [32]:
db = mysql.connector.connect(
    host='sql12.freesqldatabase.com',
    user='sql12644747',
    passwd='wtivVftCdI',
    database='sql12644747'
)
mycursor = db.cursor()

mycursor.execute("""
select * from attendence
""")
result = mycursor.fetchall()

mycursor.close()
db.close()

pd.DataFrame(result)

,0,1
0,John,12345
1,Jane,67890
2,Abhash,2147483647
3,Sudeep,6969
